# convert_raw

This notebook applies various calibration values and conversion equations to the raw FlowMow2 data.

#### Get UTM northing and easting values for nav
Add UTM-9 meters x and y to nav and save to nav_converted.h5

In [ ]:
import pandas as pd
nav = pd.read_hdf('../data/interim/nav_raw.h5', 'table')

In [ ]:
from pyproj import Proj
p = Proj(proj='utm', zone=9)

In [ ]:
nav['x'], nav['y'] = p(nav.lon.values, nav.lat.values)

In [ ]:
nav.to_hdf('../data/interim/nav_converted.h5', 'table', append=False, data_columns=True)

#### Convert Paros values to temperature and pressure

Paros pressure values are in psia, which according to the [Paros manual](../docs/G8203_Digiquartz_Broadband_Pressure_Transducers_and_Depth_Sensors_with_Frequency_Outputs.pdf) can be converted to pascals by multipyling by 9806.650, or to m of H2O by multiplying by 0.7030696. Temperature is in degrees C. The SBE3 is likely the better instrument to use for temperature. The instument [calibration constants](../data/info/paros_cals.yaml) are extracted directly from the raw DAT files. Results are saved to paros_converted.h5.

In [ ]:
paros = pd.read_hdf('../data/interim/paros_raw.h5', 'table')
paros.head()

In [ ]:
import yaml
import flowmow

In [ ]:
with open('../data/info/paros_cals.yaml') as f:
    paros_cals = yaml.load(f)

In [ ]:
paros['temp'], paros['pressure'] = flowmow.convert_paros(paros.eta, paros.tau, **paros_cals)
paros.head()

In [ ]:
import gc
paros.to_hdf('../data/interim/paros_converted.h5', 'table', append=False, data_columns=True)
paros = None
gc.collect();

#### Convert SBE3 values to temperature

The Seabird SBE3 [calibration constants](../data/info/sbe3_cals.yaml) are from the [official calibration sheets](../docs/SBE03_2014_cals.pdf) which also contain the conversion equations. More info on the SBE3 [here](../docs/datasheet-03plus-May15.pdf). We know that the SBE3 with serial number 2265 was on the stinger from this [image of the vehicle](../docs/IMG_4014.JPG-1.jpg). Results are saved to sbe3_converted.h5.

In [ ]:
sbe3 = pd.read_hdf('../data/interim/sbe3_raw.h5', 'table')
sbe3.head()

In [ ]:
with open('../data/info/sbe3_cals.yaml') as f:
    sbe3_cals = yaml.load(f)

In [ ]:
# sbe3 2265 was on the stinger and was recorded as counts_0
sbe3['temp_stinger'] = flowmow.convert_sbe3(sbe3.counts_0, **sbe3_cals[2265])
sbe3['temp_top'] = flowmow.convert_sbe3(sbe3.counts_1, **sbe3_cals[2446])
sbe3.head()

In [ ]:
sbe3.to_hdf('../data/interim/sbe3_converted.h5', 'table', append=False, data_columns=True)
sbe3 = None
gc.collect();